# TensorFlow Tutorial - 4. MLP ( Multi Layer Perceptron )

본 문서는 TensorFlow 를 사용하여 Deep Learning을 구현하기 위한 기초적인 실습 자료이다.

The code and comments are written by Dong-Hyun Kwak <imcomking@gmail.com><br>
Upgraed to Tensorflow v1.9 by NamJungGu <nowage@gmail.com> 

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.





In [1]:
import tensorflow as tf

tf.reset_default_graph()

# MNIST 데이터 불러오기

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [2]:
print(mnist.train.images.shape,mnist.train.labels.shape)
print(mnist.train.labels[1])

for i in range(28):
    s=''
    for j in range(28):
        if mnist.train.images[1][i*28+j] >0.9 :
            s=s+'O'
        else:
            s=s+' '    
    print(s)            


(55000, 784) (55000, 10)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
                            
                            
                            
                            
                            
               OOO          
             OOOOOO         
           OOOO    O        
           OOO     OO       
          O        OO       
               O   OO       
                  OOO       
               OOOOO        
            OOOOOOO         
            OOOOO           
              OOO           
                O           
                 O          
                 O          
                 O          
     O          OO          
     O        OOO           
      OOO  OOOOO            
       OOOOOOO              
         O                  
                            
                            
                            


In [3]:
# placeholder is used for feeding data.
x = tf.placeholder("float", shape=[None, 784], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
y_target = tf.placeholder("float", shape=[None, 10], name = 'y_target') # shape argument is optional, but this is useful to debug.


# all the variables are allocated in GPU memory
W1 = tf.Variable(tf.zeros([784, 256]), name = 'W1')   # create (784 * 256) matrix
b1 = tf.Variable(tf.zeros([256]), name = 'b1')        # create (1 * 256) vector
h1 = tf.sigmoid(tf.matmul(x, W1) + b1, name = 'h1')   # compute --> sigmoid(weighted summation)

# Repeat again
W2 = tf.Variable(tf.zeros([256, 10]), name = 'W2')     # create (256 * 10) matrix
b2 = tf.Variable(tf.zeros([10]), name = 'b2')          # create (1 * 10) vector
y = tf.nn.softmax(tf.matmul(h1, W2) + b2, name = 'y')  # compute classification --> softmax(weighted summation)


# define the Loss function
cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')


# define optimization algorithm
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)



correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
# correct_prediction is list of boolean which is the result of comparing(model prediction , data)


accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
# tf.cast() : changes true -> 1 / false -> 0
# tf.reduce_mean() : calculate the mean



# create summary of parameters
tf.summary.histogram('weights_1', W1)
tf.summary.histogram('weights_2', W2)
tf.summary.histogram('y', y)
tf.summary.scalar('cross_entropy', cross_entropy)
merged = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("/tmp/mlp")


# Create Session
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) # open a session which is a envrionment of computation graph.
sess.run(tf.global_variables_initializer())# initialize the variables

# training the MLP
for i in range(5001): # minibatch iteraction
    batch = mnist.train.next_batch(100) # minibatch size
    sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1]}) # placeholder's none length is replaced by i:i+100 indexes

    if i%500 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1]})
        print ("step %d, training accuracy: %.3f"%(i, train_accuracy))



        # calculate the summary and write.
        summary = sess.run(merged, feed_dict={x:batch[0], y_target: batch[1]})
        summary_writer.add_summary(summary , i)

# for given x, y_target data set
print  ("test accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.test.images, y_target: mnist.test.labels}))
sess.close()

step 0, training accuracy: 0.130
step 500, training accuracy: 0.380
step 1000, training accuracy: 0.330
step 1500, training accuracy: 0.400
step 2000, training accuracy: 0.350
step 2500, training accuracy: 0.440
step 3000, training accuracy: 0.430
step 3500, training accuracy: 0.450
step 4000, training accuracy: 0.390
step 4500, training accuracy: 0.410
step 5000, training accuracy: 0.440
test accuracy: 0.3647
